In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import random 
import scipy.stats as stats
import matplotlib.pyplot as plt
import sys, os
import warnings
import processing_module as pm
import scipy.stats as stats
from tqdm import tqdm

warnings.filterwarnings("ignore")

<span style="color:orange">
    
## Preferential attachment ##

In [2]:
def gen_graph_PA(G_input: nx.Graph, num_nodes: int, num_neigh: int) -> nx.Graph:
    """
    Функция генерации вершин графа методом предпочтительного присоединения.
    Используются библиотеки tqdm, networkx.
    
    Input:
        G_input - изначальный неориентированный граф к которому присоединяются новые вершины.
        num_nodes - количество присоединяемых вершин.
        num_neigh - количество соседий, к которым присоединяется новая вершина.
        
    Output:
        Изначальный неориентированный граф к которому присоеденены вершины.
    """
    i_start = len(list(G_input.nodes))
    print(i_start)
    for i in tqdm(range(i_start + 1, i_start + num_nodes)):
        k = []
        for i1 in range(num_neigh):
            k.append((random.choices(list(dict(G_input.degree()).keys()), weights=list(dict(G_input.degree()).values())))[0])
        for i1 in range(num_neigh):
            G_input.add_edge(k[i1], i)     
    return G_input

In [3]:
"""
Ячейка отвечает за генерацию и тестирование графа.
"""

data_pa = {
    'Source': [1, 2, 3],
    'Target': [2, 3, 1],
}

row_labels = [1, 2, 3]

PA_graph = pd.DataFrame(data=data_pa, index=row_labels)

G_pa = nx.Graph()
G_pa.add_edges_from(PA_graph.values.tolist())


num_neigh = 5
num_nodes = 100


if (False):
    G_pa_1 = gen_graph_PA(G_pa, num_nodes, num_neigh)
    print(G_pa_1)

<span style="color:orange">
    
## Clustering attachment ##

In [4]:
def gen_graph_CA(G_input: nx.Graph, num_nodes: int, num_neigh: int) -> nx.Graph:
    """
    Функция генерации вершин графа методом кластерного присоединения.
    Используются библиотеки tqdm, networkx.
    Содержит циклы так как k.append((random.choices)) может выдавать 2 одинаковых числа. 
    
    Input:
        G_input - изначальный неориентированный граф к которому присоединяются новые вершины.
        num_nodes - количество присоединяемых вершин.
        num_neigh - количество соседий, к которым присоединяется новая вершина.
        
    Output:
        Изначальный неориентированный граф к которому присоеденены вершины.
    """
    i_start = len(list(G_input.nodes))
    for i in tqdm(range(i_start + 1, i_start + num_nodes)):
        local_calst = list(nx.clustering(G_input).values())
        for j in range(len(local_calst)):
            local_calst[j] += 0.001
        k = []
        for i1 in range(num_neigh):
            k.append((random.choices(list(nx.clustering(G_input).keys()), weights=local_calst))[0])
        for i1 in range(num_neigh):
            G_input.add_edge(k[i1], i)
    return G_input

In [5]:
"""
Ячейка отвечает за генерацию и тестирование графа.
"""

data_ca = {
    'Source': [1, 2, 3],
    'Target': [2, 3, 1],
}

row_labels = [1, 2, 3]

CA_graph = pd.DataFrame(data=data_ca, index=row_labels)

G_ca = nx.Graph()
G_ca.add_edges_from(CA_graph.values.tolist())

num_neigh = 5
num_nodes = 100


if (False):
    G_ca_1 = gen_graph_CA(G_ca, num_nodes, num_neigh)
    print(G_ca_1)

In [6]:
if (False):
    my_path = os.path.abspath("") + "/.."
    (nx.to_pandas_edgelist(G_ca)).to_csv(my_path + "/Data_postproc" + '/CA_graph.csv', index=False)

<span style="color:orange">
    
## Alpha Betta Gamma scheme ##

In [7]:
"""

МОДУЛЬ ГЕНЕРАЦИИ ГРАФОВ 

Включает в себя:
    -gen_graph_PA
    -gen_graph_CA
    -gen_graph_ABG
"""


# def gen_graph_ABG(G_input: nx.DiGraph, num_iterations: int, alpha: float, beta: float, \
#                   d_in: float, d_out: float) -> nx.DiGraph:
#     """
#     Функция генерации вершин графа методом альфа, бетта, гамма присоединения.
#     Используются библиотеки tqdm, networkx.
#     gamma = 1 - (alpha + beta).
    
#     Input:
#         G_input - изначальный ориентированный граф к которому присоединяются новые вершины.
#         num_iterations - количество итераций алгоритма.
#         alpha - float вероятность.
#         beta - float вероятность. 
#         d_in - float.
#         d_out - float.
        
#     Output:
#         Изначальный граф к которому присоеденены вершины.
#     """
#     assert ((alpha + beta) <= 1)

#     i_start = len(list(G_input.nodes))
#     gamma = 1 - alpha - beta
#     print(alpha, beta, gamma, G_input)

#     for i in tqdm(range(0, num_iterations + 1)):
#         iter_prob = random.choices([1, 2, 3], weights=[alpha, beta, gamma])[0]
        
#         I_n1 = list(dict(G_input.in_degree()).values())
#         O_n1 = list(dict(G_input.out_degree()).values())
        
#         N_I = list(dict(G_input.in_degree()).keys())
#         N_O = list(dict(G_input.out_degree()).keys())
        
#         N = len(I_n1)
#         n = sum(list(dict(G_input.in_degree()).values()))
#         u = list(G_input.nodes)[-1] + 1
#         num = len(I_n1) + 1

#         assert (N_I == N_O)
        
#         if (iter_prob == 1):
#             denominator = n - 1 + d_in * N
#             local_eq = []
#             for j in range(len(I_n1)):
#                 local_eq.append((I_n1[j] + d_in) / denominator)
#             w = (random.choices(N_I, weights=local_eq))[0]
#             G_input.add_edge(num, w)

#         if (iter_prob == 2):
#             denominator_1 = n - 1 + d_in * N
#             denominator_2 = n - 1 + d_out * N
#             local_eq = []
#             for j in range(len(I_n1)):
#                 local_eq.append(((I_n1[j] + d_in) / denominator_1) * ((O_n1[j] + d_out) / denominator_2))
#             w = (random.choices(N_I, weights=local_eq))[0]
#             u = (random.choices(N_I, weights=local_eq))[0]
#             G_input.add_edge(u, w)
        
#         if (iter_prob == 3):
#             denominator = n - 1 + d_out * N
#             local_eq = []
#             for j in range(len(O_n1)):
#                 local_eq.append((O_n1[j] + d_out) / denominator)
#             w = (random.choices(N_O, weights=local_eq))[0]
#             G_input.add_edge(w, num)

#     return G_input

def gen_graph_ABG(G_input: nx.DiGraph, num_iterations: int, alpha: float, beta: float, \
                  d_in: float, d_out: float) -> nx.DiGraph:
    """
    Функция генерации вершин графа методом альфа, бетта, гамма присоединения.
    Используются библиотеки tqdm, networkx.
    gamma = 1 - (alpha + beta).
    
    Input:
        G_input - изначальный ориентированный граф к которому присоединяются новые вершины.
        num_iterations - количество итераций алгоритма.
        alpha - float вероятность.
        beta - float вероятность. 
        d_in - float.
        d_out - float.
        
    Output:
        Изначальный граф к которому присоеденены вершины. 
        Без self loops
    """
    assert ((alpha + beta) <= 1)

    i_start = len(list(G_input.nodes))
    gamma = 1 - alpha - beta
    print(alpha, beta, gamma, G_input)

    for i in tqdm(range(0, num_iterations + 1)):
        iter_prob = random.choices([1, 2, 3], weights=[alpha, beta, gamma])[0]
        
        I_n1 = list(dict(G_input.in_degree()).values())
        O_n1 = list(dict(G_input.out_degree()).values())
        
        N_I = list(dict(G_input.in_degree()).keys())
        N_O = list(dict(G_input.out_degree()).keys())
        
        N = len(I_n1)
        n = sum(list(dict(G_input.in_degree()).values()))
        u = list(G_input.nodes)[-1] + 1

        assert (N_I == N_O)
        
        if (iter_prob == 1):
            denominator = n - 1 + d_in * N
            local_eq = []
            for j in range(len(I_n1)):
                local_eq.append((I_n1[j] + d_in) / denominator)
            w = (random.choices(N_I, weights=local_eq))[0]
            G_input.add_edge(i, w)

        if (iter_prob == 2):
            denominator_1 = n - 1 + d_in * N
            denominator_2 = n - 1 + d_out * N
            local_eq = []
            for j in range(len(I_n1)):
                local_eq.append(((I_n1[j] + d_in) / denominator_1) * ((O_n1[j] + d_out) / denominator_2))
            w = (random.choices(N_I, weights=local_eq))[0]
            u = (random.choices(N_I, weights=local_eq))[0]
            if (u != w):
                G_input.add_edge(u, w)
        
        if (iter_prob == 3):
            denominator = n - 1 + d_out * N
            local_eq = []
            for j in range(len(O_n1)):
                local_eq.append((O_n1[j] + d_out) / denominator)
            w = (random.choices(N_O, weights=local_eq))[0]
            G_input.add_edge(w, i)

    return G_input


In [8]:
"""
Ячейка отвечает за генерацию и тестирование графа.
"""

data_abg = {
    'Source': [1, 2, 3],
    'Target': [2, 3, 1],
}

row_labels = [1, 2, 3]

ABG_graph = pd.DataFrame(data=data_abg, index=row_labels)

G_abg = nx.DiGraph()
G_abg.add_edges_from(ABG_graph.values.tolist())

num_nodes = 3

if (False):
    G_abg_1 = gen_graph_ABG(G_abg, num_iterations = 10000, alpha = 0.5, beta = 0.4, d_in = 0.1, d_out = 0.1)

<span style="color:orange">
    
## Graph generation ##

In [9]:
def gen_save_CA(G_input: nx.Graph, num_nodes: int, num_neigh: int, blocks: int, folder_name: str) -> nx.Graph:
    """
    Генерация графа до половины времени по схеме CA.
    С автоматическим сохранением снимков графа.
    """
    my_path = os.path.abspath("") + "/.."
    for t in range(0, blocks):
        G_input = gen_graph_CA(G_input, num_nodes, num_neigh)
        # pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
        (nx.to_pandas_edgelist(G_input)).to_csv(my_path + "/Data_postproc/" + folder_name + "/Graph_CA" \
                                               + str(t) + "_part.csv", index=False)
    return G_input



def gen_save_PA(G_input: nx.Graph, num_nodes: int, num_neigh: int, blocks: int, folder_name: str) -> nx.Graph:
    """
    Генерация графа до половины времени по схеме PA.
    С автоматическим сохранением снимков графа.
    """
    my_path = os.path.abspath("") + "/.."
    for t in range(0, blocks):
        G_input = gen_graph_PA(G_input, num_nodes, num_neigh)
        # pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
        (nx.to_pandas_edgelist(G_input)).to_csv(my_path + "/Data_postproc/" + folder_name + "/Graph_PA" \
                                               + str(t) + "_part.csv", index=False)
    return G_input


def gen_save_ABG(G_input: nx.DiGraph, blocks: int, folder_name: str, param: list) -> nx.DiGraph:
    """
    Генерация графа до половины времени по схеме ABG.
    С автоматическим сохранением снимков графа.
    param = [num_iterations, alpha, beta, d_in, d_out]
    """
    
    num_iterations = param[0]
    alpha = param[1]
    beta = param[2]
    d_in = param[3]
    d_out = param[4]
    my_path = os.path.abspath("") + "/.."
    for t in range(0, blocks):
        G_input = gen_graph_ABG(G_input, num_iterations, alpha, beta, d_in, d_out)
        # pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
        (nx.to_pandas_edgelist(G_input)).to_csv(my_path + "/Data_postproc/" + folder_name + "/Graph_ABG" \
                                               + str(t) + "_part.csv", index=False)
    return G_input


def load_graph_pr(blocks: int, folder_name: str, gen_name: str) -> list:
    """
    Загрузка цепочек данных по пейджрангам по определённым блокам.
    """
    for t in range(0, blocks):
        my_path = os.path.abspath("") + "/.."
        CA_graph_df = pd.read_csv(my_path + "/Data_postproc/" + folder_name + "/Graph" + gen_name \
                                                   + str(t) + "_part.csv")
        G_init = nx.from_pandas_edgelist(CA_graph_df)
        pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
    return pr


# def load_graph(blocks: int, folder_name: str, gen_name: str) -> nx.Graph:
#     """
#     Загрузка графа.
#     """
#     for t in range(0, blocks):
#         my_path = os.path.abspath("") + "/.."
#         CA_graph_df = pd.read_csv(my_path + "/Data_postproc/" + folder_name + "/Graph" + gen_name \
#                                                    + str(t) + "_part.csv")
#         G_init = nx.from_pandas_edgelist(CA_graph_df)
#         pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
#     return pr


def load_graph_di(t: int, folder_name: str, gen_name: str) -> nx.DiGraph:
    """
    Загрузка направленного графа.
    """
    my_path = os.path.abspath("") + "/.."
    CA_graph_df = pd.read_csv(my_path + "/Data_postproc/" + folder_name + "/Graph" + gen_name \
                                                   + str(t) + "_part.csv")
    G_init = nx.from_pandas_edgelist(CA_graph_df, source='source', target='target', \
                                   create_using=nx.DiGraph())
    return G_init

In [71]:
"""
Ячейка отвечает за генерацию и тестирование графа.
"""

data_abg = {
    'Source': [1, 2, 3],
    'Target': [2, 3, 1],
}

row_labels = [1, 2, 3]

ABG_graph = pd.DataFrame(data=data_abg, index=row_labels)

G_abg = nx.DiGraph()
G_abg.add_edges_from(ABG_graph.values.tolist())


blocks = 7
pr = []
# param = [5000, 0.2, 0.5, 0.9, 0.9]
param = [15000, 0.07, 0.86, 0.95, 0.95] #что то интересное
# num_iterations = 10000, alpha = 0.5, beta = 0.4, d_in = 0.1, d_out = 0.1

folder_name = "ABG_graph_2"


gen_save_ABG(G_abg, blocks, folder_name, param)

0.07 0.86 0.06999999999999995 DiGraph with 3 nodes and 3 edges


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15001/15001 [00:25<00:00, 583.26it/s]


0.07 0.86 0.06999999999999995 DiGraph with 2080 nodes and 4087 edges


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15001/15001 [01:16<00:00, 197.23it/s]


0.07 0.86 0.06999999999999995 DiGraph with 3950 nodes and 7417 edges


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15001/15001 [03:34<00:00, 69.92it/s]


0.07 0.86 0.06999999999999995 DiGraph with 7943 nodes and 16293 edges


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15001/15001 [04:18<00:00, 58.03it/s]


0.07 0.86 0.06999999999999995 DiGraph with 8912 nodes and 19092 edges


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15001/15001 [04:42<00:00, 53.19it/s]


<span style="color:orange">
    
## Graph analysis CA + PA ##

In [ ]:
print(len(pr[19]))

In [11]:
x_gr = list()
y_gr = list()
for i in range(blocks):
    x = pm.hill(pr[i])
    eye_val = (pm.eye_ball(x))[0]
    x_gr.append(i)
    y_gr.append(eye_val)
    print(i, eye_val)

NameError: name 'blocks' is not defined

In [ ]:
plt.plot(x_gr, y_gr, marker='o', markersize=7)
plt.xlabel('Временной шаг') #Хвостовые ингде
plt.ylabel('EVI') #Подпись для оси y
plt.title('Зависимость EVI от временного шага') #Название
plt.show()

In [ ]:
med = []
for i in range(30):
    med.append(np.median(pr[i]))

plt.plot(x_gr, med, marker='o', markersize=7)
plt.xlabel('Временной блок') #Хвостовые ингде
plt.ylabel('Медиана данных в блоке') #Подпись для оси y
plt.title('Медиана данных в блоке от времени') #Название
plt.show()

In [ ]:
def phillips_loretan(x1, x2):
    """
    Функция теста phillips_loretan для двух последовательностей
    """
    tail_ind_1, tail_num_1 = pm.eye_ball(pm.hill(x1))
    tail_ind_2, tail_num_2 = pm.eye_ball(pm.hill(x2))
    s = (tail_num_1*(tail_ind_2)**2 * (tail_ind_1/tail_ind_2 - 1)**2)/ \
    (tail_ind_1**2 + (tail_num_1/tail_num_2)*tail_ind_2**2)
    return s

In [12]:
"""
Тесты Филлипса Лоретана для разных блоков.
"""

group_1 = []
for i in range(0, 10):
    for j in range(0, 10):
        # print(i, j, phillips_loretan(pr[i], pr[j]))
        group_1.append(phillips_loretan(pr[i], pr[j]))

group_1_2 = []
# print('\n')
for i in range(0, 10):
    for j in range(10, 20):
        # print(i, j, phillips_loretan(pr[i], pr[j]))
        group_1_2.append(phillips_loretan(pr[i], pr[j]))

group_2 = []
# print('\n')
for i in range(10, 20):
    for j in range(10, 20):
        # print(i, j, phillips_loretan(pr[i], pr[j]))
        group_2.append(phillips_loretan(pr[i], pr[j]))

print('group_1 \n', min(group_1), max(group_1), '\n')
print('group_2 \n', min(group_2), max(group_2), '\n')
print('group_1_2 \n', min(group_1_2), max(group_1_2), '\n')

NameError: name 'phillips_loretan' is not defined

In [13]:
phillips_loretan(pr[14], pr[15])

NameError: name 'phillips_loretan' is not defined

<span style="color:orange">
    
## Graph analysis ABG ##

In [83]:
folder_name = "ABG_graph_2"
gen_name = '_ABG'
t = 4

G_ABG_1 = load_graph_di(t, folder_name, gen_name)

In [84]:
print(len(G_ABG_1.nodes()))

7943


In [98]:
print('graph analysis')
print(G_ABG_1)

G_ABG_und = G_ABG_1.to_undirected()

comm = nx.community.louvain_communities(G_ABG_und, resolution = 0.8)
comm = sorted(comm, key=len, reverse=True)


print('\n \n \n degree analysis')
in_data = pd.DataFrame(data = G_ABG_1.in_degree(), columns=['node', 'node_degree'])
print(in_data.sort_values(by = 'node_degree', ascending = False)[0:20])


print('\n \n \n Comm len analysis')
print('all communities:', len(comm))
for i in range(10):
    # if (len(comm[i]) > 0.05 * len(G_ABG_1.nodes())):
    #     print(i, len(comm[i]))
    print(i, len(comm[i]))
# print(comm[1])

print(' \n \n \n Comm separeted analysis')
in_data = pd.DataFrame(data = G_ABG_1.in_degree(), columns=['node', 'node_degree'])
for i in range(5):
    ser_comm = pd.DataFrame(data=list(comm[i]), columns=['node'])
    print('\n', i)
    print(pd.merge(ser_comm, in_data).sort_values(by = 'node_degree', ascending = False)[0:5])

graph analysis
DiGraph with 7943 nodes and 16293 edges

 
 
 degree analysis
    node  node_degree
7     16          549
23    54          508
5     45          501
0      1          382
2      3          358
4     42          350
3     10          283
1      2          153
17    79          136
26    66          101
18   309           97
21   139           97
53   217           92
46    61           86
35    72           73
14    96           73
47   305           67
39   165           62
10   137           57
32   414           56

 
 
 Comm len analysis
all communities: 35
0 804
1 668
2 665
3 629
4 557
5 520
6 510
7 409
8 362
9 341
 
 
 
 Comm separeted analysis

 0
     node  node_degree
60    139           97
134   309           97
28     61           86
34     72           73
42     96           73

 1
     node  node_degree
15     54          508
155   428           26
440  1232           22
79    212           20
292   819           18

 2
     node  node_degree
2      16      

<span style="color:orange">
    
## Old code ##

In [ ]:
# for t in range(0, blocks//2):
#     G_init = gen_graph_CA(G_init, num_nodes, num_neigh)
#     pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]
#     (nx.to_pandas_edgelist(G_init)).to_csv(my_path + "/Data_postproc" + "/Big_graph" + "/CA_PA_" \
#                                                + str(t) + "_nodes" + ".csv", index=False)


# for t in range(0, blocks//2):
#     my_path = os.path.abspath("") + "/.."
#     CA_graph_df = pd.read_csv(my_path + "/Data_postproc" + "/Big_graph" + "/CA_PA_" \
#                                                + str(t) + "_nodes" + ".csv")
#     G_init = nx.from_pandas_edgelist(CA_graph_df)
#     pr[t] = list((nx.pagerank(G_init)).values())[-num_nodes:-1]

In [6]:
# for i in range(len(edges_ca['source'])):
#     print(i, edges_ca[edges_ca['source'] == i].size)
# print(edges_ca[edges_ca['source'] == 1])

In [7]:
# data = {
#     'Source': [1, 2],
#     'Target': [2, 1],
#     'Node_degree': [1, 1]
# }

# row_labels = [1, 2]

# PA_graph = pd.DataFrame(data=data, index=row_labels)

# flg_PA = 0

# l = []
# if(flg_PA == 1):
#     for i in tqdm(range(3, 10000)):
#         k = (random.choices(PA_graph['Source'].values, weights=PA_graph['Node_degree'].values))[0]
#         if (i == 3):
#             l = [PA_graph['Source'].values, PA_graph['Node_degree'].values]
#         # print(k)
#         PA_graph.loc[i] = [i, k, 1]
#         PA_graph['Node_degree'][k] += 1

# print(l)

In [49]:
# генерация графа CA старая версия
# 
# if(flg_CA == 1):
#     for i in tqdm(range(4, num_nodes_ca)):
#         local_calst = list(nx.clustering(G_ca).values())
#         for j in range(len(local_calst)):
#             local_calst[j] += 0.001
#         k = []
#         for i1 in range(num_neigh):
#             k.append((random.choices(list(nx.clustering(G_ca).keys()), weights=local_calst))[0])
#         for i1 in range(num_neigh):
#             G_ca.add_edge(k[i1], i)
